# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx
from functools import reduce

#not currently using jellyfish
import jellyfish as jf

## Set up the Dataframes

In [ ]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

In [ ]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
dfs = []
for key in sourceFiles.keys():
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])

In [ ]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)

In [ ]:
df_sports247 = pandas.DataFrame(sports247).set_index('ID')
df_rivals = pandas.DataFrame(rivals).set_index('ID')
df_sports247.index.name = '247_ID'
df_sports247.insert(0, 'ID', df_sports247.index)
df_rivals.index.name = 'rivals_ID'
df_rivals.insert(0, 'ID', df_rivals.index)

## Import the Pickle file

In [ ]:
features = pandas.read_pickle("final_review.pkl")

## Convert the 247 Dataframe to a dictionary 
> Pushes the dataframe index into a column value so I can leverage it in future steps

In [ ]:
df_sports247.reset_index(inplace=True)
sportDict = df_sports247.to_dict('records')

## This is the brain
> This is what joins records from both 247 and Rivals based on the linking output.  It uses 247 as master, so automatically returns all of those records and then marks the rivals corresponding record that is above the confidence threshold set in the previous script.

####  Few things to fix:
1. The script feeding in this data no longer returns multiple rows for a given 247 ID.  I've updated that script so that it only returns the max value.  So a lot of the checks around confidence and max values can be removed.
2. There has to be a more elegant way to do this.

In [ ]:
finalSet = []
for record in sportDict:
    
    finalRecord = {}
    for key,value in record.items():
        finalRecord[key] = value
    
    try:
        multiResults = features.loc[features['247_ID'] == record['247_ID']]
        rivalsID = multiResults['rivals'].values
        
        if (multiResults['ID'].max() == 1):
            finalRecord['confidence'] = 1
        else:
            finalRecord['confidence'] = multiResults['sum'].max()
    
    except Exception as e:
        finalRecord['error'] = 'No feature found with that ID'

    try:
        rivalsResult = (df_rivals.loc[df_rivals['ID'] == rivalsID[0]]).to_dict()
        
        for key, value in rivalsResult.items():
            finalRecord['r_' + key] = value[rivalsID[0]]
    
    except Exception as e:
        finalRecord['error'] = 'No rivals match found'
    
    finalSet.append(finalRecord)

In [ ]:
df_rivals.reset_index(inplace=True)
rivalsDict = df_rivals.to_dict('records')

count = 0
for rival in rivalsDict:
    found = 0
    
    for record in finalSet:
        if ('r_ID' in record.keys() and rival['ID'] == record['r_ID']):
            found = 1

    if (found == 0):
        newPlayer = {}
        for key, value in rival.items():
            newPlayer['r_' + key] = value
        newPlayer['school'] = rival['school']
        newPlayer['year'] = rival['year']
        print(newPlayer)
        count = count + 1
        finalSet.append(newPlayer)

print(count)

{'r_rivals_ID': 'darrylturner_maryland_2016', 'r_ID': 'darrylturner_maryland_2016', 'r_rawSchool': 'Maryland', 'r_school': 'maryland', 'r_year': '2016', 'r_playerName': 'darrylturner', 'r_city': 'Hyattsville', 'r_state': 'MD', 'r_highSchool': 'DeMatha', 'r_position': 'ATH', 'r_height': 69.0, 'r_weight': 188.0, 'r_stars': 4, 'r_nationalRank': nan, 'r_positionRank': 17.0, 'r_stateRank': 7.0, 'r_displayName': 'Darryl Turner', 'school': 'maryland', 'year': '2016'}
{'r_rivals_ID': 'brandonware_pennstate_2008', 'r_ID': 'brandonware_pennstate_2008', 'r_rawSchool': 'Penn State', 'r_school': 'pennstate', 'r_year': '2008', 'r_playerName': 'brandonware', 'r_city': 'Harrisburg', 'r_state': 'PA', 'r_highSchool': '', 'r_position': 'DT', 'r_height': 76.0, 'r_weight': 358.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Brandon Ware', 'school': 'pennstate', 'year': '2008'}
{'r_rivals_ID': 'stevebrown_michigan_2006', 'r_ID': 'stevebrown_michigan_2006'

{'r_rivals_ID': 'etevamauga_nebraska_2020', 'r_ID': 'etevamauga_nebraska_2020', 'r_rawSchool': 'Nebraska', 'r_school': 'nebraska', 'r_year': '2020', 'r_playerName': 'etevamauga', 'r_city': 'Pleasant Hill', 'r_state': 'CA', 'r_highSchool': 'Diablo Valley C.C.', 'r_position': 'LB', 'r_height': 74.0, 'r_weight': 210.0, 'r_stars': 4, 'r_nationalRank': 8.0, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Eteva Mauga', 'school': 'nebraska', 'year': '2020'}
{'r_rivals_ID': 'tyveljemison_purdue_2013', 'r_ID': 'tyveljemison_purdue_2013', 'r_rawSchool': 'Purdue', 'r_school': 'purdue', 'r_year': '2013', 'r_playerName': 'tyveljemison', 'r_city': 'Fort Wayne', 'r_state': 'IN', 'r_highSchool': 'Bishop Luers', 'r_position': 'DB', 'r_height': 70.0, 'r_weight': 175.0, 'r_stars': 3, 'r_nationalRank': nan, 'r_positionRank': 52.0, 'r_stateRank': 10.0, 'r_displayName': 'TyVel Jemison', 'school': 'purdue', 'year': '2013'}
{'r_rivals_ID': 'anthonymorgan_iowa_2012', 'r_ID': 'anthonymorgan_iowa_20

{'r_rivals_ID': 'chrisphillips_indiana_2004', 'r_ID': 'chrisphillips_indiana_2004', 'r_rawSchool': 'Indiana', 'r_school': 'indiana', 'r_year': '2004', 'r_playerName': 'chrisphillips', 'r_city': 'Atlanta', 'r_state': 'GA', 'r_highSchool': '', 'r_position': 'DB', 'r_height': 73.0, 'r_weight': 185.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Chris Phillips', 'school': 'indiana', 'year': '2004'}
{'r_rivals_ID': 'jahkeengilmore_indiana_2002', 'r_ID': 'jahkeengilmore_indiana_2002', 'r_rawSchool': 'Indiana', 'r_school': 'indiana', 'r_year': '2002', 'r_playerName': 'jahkeengilmore', 'r_city': 'Brooklyn', 'r_state': 'NY', 'r_highSchool': '', 'r_position': 'WR', 'r_height': 73.0, 'r_weight': 200.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': 90.0, 'r_stateRank': nan, 'r_displayName': 'Jahkeen Gilmore', 'school': 'indiana', 'year': '2002'}
{'r_rivals_ID': 'antoniogully_illinois_2006', 'r_ID': 'antoniogully_illinois_2006', 'r_rawSc

{'r_rivals_ID': 'ricardomiller_michigan_2010', 'r_ID': 'ricardomiller_michigan_2010', 'r_rawSchool': 'Michigan', 'r_school': 'michigan', 'r_year': '2010', 'r_playerName': 'ricardomiller', 'r_city': 'Ann Arbor', 'r_state': 'MI', 'r_highSchool': 'Pioneer', 'r_position': 'WR', 'r_height': 74.0, 'r_weight': 208.0, 'r_stars': 3, 'r_nationalRank': nan, 'r_positionRank': 66.0, 'r_stateRank': 12.0, 'r_displayName': 'Ricardo Miller', 'school': 'michigan', 'year': '2010'}
{'r_rivals_ID': 'jasonmccourty_rutgers_2005', 'r_ID': 'jasonmccourty_rutgers_2005', 'r_rawSchool': 'Rutgers', 'r_school': 'rutgers', 'r_year': '2005', 'r_playerName': 'jasonmccourty', 'r_city': 'Nanuet', 'r_state': 'NY', 'r_highSchool': '', 'r_position': 'RB', 'r_height': 71.0, 'r_weight': 158.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Jason McCourty', 'school': 'rutgers', 'year': '2005'}
{'r_rivals_ID': 'chasetriplett_purdue_2019', 'r_ID': 'chasetriplett_purdue_2019', '

{'r_rivals_ID': 'robertsmith_purdue_2003', 'r_ID': 'robertsmith_purdue_2003', 'r_rawSchool': 'Purdue', 'r_school': 'purdue', 'r_year': '2003', 'r_playerName': 'robertsmith', 'r_city': 'Arlington', 'r_state': 'TX', 'r_highSchool': 'Bowie', 'r_position': 'DB', 'r_height': 74.0, 'r_weight': 180.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Robert Smith', 'school': 'purdue', 'year': '2003'}
{'r_rivals_ID': 'jerzhannewton_illinois_2020', 'r_ID': 'jerzhannewton_illinois_2020', 'r_rawSchool': 'Illinois', 'r_school': 'illinois', 'r_year': '2020', 'r_playerName': 'jerzhannewton', 'r_city': 'Clearwater', 'r_state': 'FL', 'r_highSchool': 'Central Catholic', 'r_position': 'DT', 'r_height': 75.0, 'r_weight': 250.0, 'r_stars': 3, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Jerzhan Newton', 'school': 'illinois', 'year': '2020'}
{'r_rivals_ID': 'dennisziegler_indiana_2007', 'r_ID': 'dennisziegler_indiana_200

...

{'r_rivals_ID': 'barrystokes_troy_2007', 'r_ID': 'barrystokes_troy_2007', 'r_rawSchool': 'Troy', 'r_school': 'troy', 'r_year': '2007', 'r_playerName': 'barrystokes', 'r_city': 'Soperton', 'r_state': 'GA', 'r_highSchool': 'Treutlen County', 'r_position': 'DT', 'r_height': 76.0, 'r_weight': 288.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Barry Stokes', 'school': 'troy', 'year': '2007'}
{'r_rivals_ID': 'qushaunlee_arkansasstate_2010', 'r_ID': 'qushaunlee_arkansasstate_2010', 'r_rawSchool': 'Arkansas State', 'r_school': 'arkansasstate', 'r_year': '2010', 'r_playerName': 'qushaunlee', 'r_city': 'Prattville', 'r_state': 'AL', 'r_highSchool': 'Prattville', 'r_position': 'LB', 'r_height': 72.0, 'r_weight': 215.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Qushaun Lee', 'school': 'arkansasstate', 'year': '2010'}
{'r_rivals_ID': 'damianwilliams_texasstate_2017', 'r_ID': 'damianwilliams

{'r_rivals_ID': 'michaelgilmore_texasstate_2010', 'r_ID': 'michaelgilmore_texasstate_2010', 'r_rawSchool': 'Texas State', 'r_school': 'texasstate', 'r_year': '2010', 'r_playerName': 'michaelgilmore', 'r_city': 'La Marque', 'r_state': 'TX', 'r_highSchool': 'La Marque', 'r_position': 'WR', 'r_height': 69.0, 'r_weight': 170.0, 'r_stars': 0, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Michael Gilmore', 'school': 'texasstate', 'year': '2010'}
{'r_rivals_ID': 'fredbrown_arkansasstate_2003', 'r_ID': 'fredbrown_arkansasstate_2003', 'r_rawSchool': 'Arkansas State', 'r_school': 'arkansasstate', 'r_year': '2003', 'r_playerName': 'fredbrown', 'r_city': 'Dallas', 'r_state': 'TX', 'r_highSchool': 'Lincoln', 'r_position': 'ATH', 'r_height': 75.0, 'r_weight': 170.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Fred Brown', 'school': 'arkansasstate', 'year': '2003'}
{'r_rivals_ID': 'dariansmall_appalachianstate

{'r_rivals_ID': 'brandonnettles_louisianamonroe_2018', 'r_ID': 'brandonnettles_louisianamonroe_2018', 'r_rawSchool': 'UL-Monroe', 'r_school': 'louisianamonroe', 'r_year': '2018', 'r_playerName': 'brandonnettles', 'r_city': 'Mansfield', 'r_state': 'TX', 'r_highSchool': 'Mansfield', 'r_position': 'DE', 'r_height': 74.0, 'r_weight': 226.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Brandon Nettles', 'school': 'louisianamonroe', 'year': '2018'}
{'r_rivals_ID': 'marklegree_appalachianstate_2007', 'r_ID': 'marklegree_appalachianstate_2007', 'r_rawSchool': 'Appalachian St', 'r_school': 'appalachianstate', 'r_year': '2007', 'r_playerName': 'marklegree', 'r_city': 'Columbus', 'r_state': 'GA', 'r_highSchool': 'Pacelli Catholic', 'r_position': 'DB', 'r_height': 72.0, 'r_weight': 180.0, 'r_stars': 0, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Mark LeGree', 'school': 'appalachianstate', 'year': '2007'}
{

{'r_rivals_ID': 'sherrodmartin_troy_2003', 'r_ID': 'sherrodmartin_troy_2003', 'r_rawSchool': 'Troy', 'r_school': 'troy', 'r_year': '2003', 'r_playerName': 'sherrodmartin', 'r_city': 'Griffin', 'r_state': 'GA', 'r_highSchool': 'Griffin', 'r_position': 'DB', 'r_height': 72.0, 'r_weight': 180.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Sherrod Martin', 'school': 'troy', 'year': '2003'}
{'r_rivals_ID': 'adrianbellard_texasstate_2012', 'r_ID': 'adrianbellard_texasstate_2012', 'r_rawSchool': 'Texas State', 'r_school': 'texasstate', 'r_year': '2012', 'r_playerName': 'adrianbellard', 'r_city': 'Brenham', 'r_state': 'TX', 'r_highSchool': 'Brenham', 'r_position': 'OL', 'r_height': 77.0, 'r_weight': 295.0, 'r_stars': 3, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Adrian Bellard', 'school': 'texasstate', 'year': '2012'}
{'r_rivals_ID': 'olufemiodaibo_troy_2013', 'r_ID': 'olufemiodaibo_troy_2013', 'r_ra

{'r_rivals_ID': 'robertboone_texasstate_2005', 'r_ID': 'robertboone_texasstate_2005', 'r_rawSchool': 'Texas State', 'r_school': 'texasstate', 'r_year': '2005', 'r_playerName': 'robertboone', 'r_city': 'Allen', 'r_state': 'TX', 'r_highSchool': '', 'r_position': 'ATH', 'r_height': 69.0, 'r_weight': 185.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Robert Boone', 'school': 'texasstate', 'year': '2005'}
{'r_rivals_ID': 'jomaulmason_texasstate_2002', 'r_ID': 'jomaulmason_texasstate_2002', 'r_rawSchool': 'Texas State', 'r_school': 'texasstate', 'r_year': '2002', 'r_playerName': 'jomaulmason', 'r_city': 'Brenham', 'r_state': 'TX', 'r_highSchool': 'Blinn C.C.', 'r_position': 'OL', 'r_height': 72.0, 'r_weight': 270.0, 'r_stars': 2, 'r_nationalRank': nan, 'r_positionRank': nan, 'r_stateRank': nan, 'r_displayName': 'Jomaul Mason', 'school': 'texasstate', 'year': '2002'}
{'r_rivals_ID': 'dominicpadrta_arkansasstate_2005', 'r_ID': 'dominicpadrt

## Insert the list of dictionaries back into a dataframe

In [ ]:
df_finalSet = pandas.DataFrame(finalSet)

## Create a new ID field without Year
> This was tricky, the other data sets can't use 'year' as an identifer because their year field reflect when an achievement was reached for any given player - when they were drafted or named all conference, etc.

In [ ]:
df_finalSet['ID'] = df_finalSet['ID'].str[:-5]
df_finalSet.to_csv('review.csv')  

## Strictly Link to the other datasets

In [ ]:
summarizedDir = '..//summarizedData//'

## Load the source file dict
summarizedFiles = json.loads(open('..//config//summarizedFiles.json', "r").read())

## This needs to be cleaned up.  
>I don't want everything coming back as a key because I don't need 247 or Rivals here.

In [ ]:
dfs = []
keys = ['allConf','allAmerican','nflData','ncaa']
for key in keys:
    print(key)
    vars()[key] = pandas.DataFrame(json.loads(open(summarizedDir + summarizedFiles[key][0], "r", encoding="utf-8").read()))
    dfs.append(vars()[key])
    

allConf
allAmerican
nflData
ncaa


## Manually adding the 247/Rivals dataset

In [ ]:
dfs.insert(0, df_finalSet)

## Magical joining on ID

In [ ]:
df_final = reduce(lambda left,right: pandas.merge(left,right,how="left", on='ID'), dfs)
df_final.head()

,247_ID,ID,school,year,playerName,highSchool,city,state,position,height,...,draft_pick,team_y,pos,all_pros_first_team,pro_bowls,years_as_primary_starter,g,draft_year,ncaa_gamesPlayed,ncaa_gamesStarted
0,paulthurston_nebraska_2012,paulthurston_nebraska,nebraska,2012,paulthurston,Arvada West,Arvada,CO,OT,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gregmcmullen_nebraska_2012,gregmcmullen_nebraska,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,SDE,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,imanicross_nebraska_2012,imanicross_nebraska,nebraska,2012,imanicross,North Hall,Gainesville,GA,RB,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,nebraska,2012,tommyarmstrong,Steele,Cibolo,TX,DUAL,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
with open("..//linkedPlayers.csv", "w", encoding="utf-8") as write_file:
                write_file.write(df_final.to_csv())